<a href="https://colab.research.google.com/github/gsanahi/EjerciciosFernando/blob/main/Copia_de_5_map_y_reduce_juntos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajando Juntos: map, reduce y filter

![](https://2.bp.blogspot.com/-QFj3KGiRAAI/TYqMl92KOtI/AAAAAAAADnU/v4WS_SI0vvs/s1600/mapreduce.png)



## Gestión de un restaurante de comida rápida

Supongamos que mientras te preparas para entrar en le mercado de la programación, te contratan para llevar un restaurante de comida rápida. El producto final es una hamburguesa y lleva los siguientes ingredientes:

* pan
* loncha de tomate
* loncha de pepinillo
* loncha de cebolla
* carne

Es decir, cada producto final es la combinación de 5 ingredientes, cada uno de los cuales tiene que ser procesado previamente. Como sabes programación, el problema de cómo optimizar la producción de hamburguesas está chupado. 

Preparas 6 mesas. Cinco para los ingredientes y una final. Cada mesa está atendida por un empleado.

* Mesa del pan: hay una lista de panes que el empleado transforma en una lista de panes cortados por la mitad.
* Mesa del tomate: lista de tomates que el empleado transforma en una lista de lonchas
* Mesa del pepinillo: lista de pepinillos que el empleado transforma en una lista de lonchas
* Mesa de la cebolla: lista de cebollas que el empleado transforma en una lista de lonchas
* Mesa de la carne: una lista de bolas de carne molida que el empleado da forma y prepara a la plancha y transforma en una lista... ya lo habrás pillado. ;-)

En la última mesa, el empleado hace lo siguiente: va pillando un ingrediente de cada una de las otras mesas, y ensambla, o *reduce* (lo pillas, ¿verdad?) esos 5 ingredientes, a 1 hamburguesa lista. Su resultado final es una lista de hamburguesas.



Cada una de las 5 primeras mesas es una lista que se transforma en otra (ingrediantes crudos -> ingredientes preparados). El empleado correspondiente, se pasea por la mesa, produciendo la nueva lista.

La mesa final es una lista de ingredientes finales. El empleado va a cada una de las mess de ingredientes, pilla uno, y forma (en su mesa) una lista de ingredientes preparados. Luego se mueve por la mesa, reduciendo esos ingredientes al producto final.



## Puntos Importantes

Ya te habrás dado cuenta que hemso puesto en práctica nuestros conocimientos de computación para crear la maquinaria perfecta para producir hambuguesas. Algunas cosas qu ehay que destacar:

* Las mesas `map` pre-procesan los ingredientes / datos
* La mesa `reduce` aprovecha el trabajo hecho por las `map` y combina sus resultados en un producto final.

Algo *muy imporante* que tal vez no hayas visto, pero que es clave para la eficiencia del sistema es el siguiente:

> Las mesas NO dependen la una de la otra

Esto quiere decir que:

* Mientras no se queden sin ingredientes, el que corta el pan no tiene que esperar que el del tomate termine, etc...
* Mientras no se queden sin ingredientes, se pueden producir pequeños atrasos en una mesa sin que el sistema pare. Es decir, el de lso pepinillos puede ir a mear sin que se pare todo (hay suficientes penillos cortados en la lista de pepinillos cortados como para aguantar un poco)
* Cada mesa puede ser atendida por personas diferentes
* Esas personas, mientras cada una haga su trabajo, no necesitan comunicarse ni ponerse de acuerdo en nada.

----------------------------

## Ejemplo con Código

Te contratan en una empresa que vende pantuflas calefactadas por usb para programadores. Las ventas mensuales vienen desglosadas por divisa (se vende en USD, EUR, GBP y JPI) . Todo ello viene en una lista, cuyos elementos son listas con las ventas en dichas divisas y en ese orden:

In [1]:
sales = [[2300, 345, 1949, 2222, 5939], [5252, 7886, 6363, 4432, 6653], [6262626, 47774, 72727,636363], [363636, 4828, 199333, 7264]]

Transfórmalo en  una venta de totales, usando tu propio map, reduce y filter

In [3]:
def my_filter(elements, predicate):
    """
    recibe una lista y un predicado. devuelve otra lista con aquellos elementos
    que superan el test del predicado
    """
    accum = [] 
    for element in elements:
        if predicate(element):
            accum.append(element)
    return accum

def my_map(seq,transformer):
    new_seq = []
    for element in seq:
        new_seq.append(transformer(element))
    return new_seq

def my_reduce(seq, initial_value, combinator):
    """
    Recorre una lista llamada seq y va combinando los valores mediante un 
    combinador (que no sabemos muy bien qué es, pero que se recibe como parámetro. 
    Devuelve el valor reducido o combinado de todos los elementos de la lista.
    """
    accum = initial_value
    for element in seq:
        accum = combinator(accum, element) 
    return accum

In [11]:
def make_sub_totals(list_of_sales):
    """
    devuelve una lista con los totales en su propia divisa
    """
    return my_map(list_of_sales, lambda list_of_nums : my_reduce(list_of_nums, 0, lambda a,b : a+ b))

def convert_to_usd(subtotals,rates):
    """
    convierte una lista de USD,EUR,GBP , JYI
    """
    #recorro la lista de totales, multiplico cada uno por su taza de conversion correspondiente. Guardo el total en USD en la lista q estoy creando
    values_in_usd = []
    for i, subtotal in enumerate(subtotals):
      values_in_usd.append(subtotal * rates[i])
    return values_in_usd

def grand_total_usd(sales):
    """
    Devuelve un total en USD
    """
    return my_reduce(sales, 0, lambda a,b : a + b)



In [15]:
make_sub_totals(sales)
sub_totals_in_usd = convert_to_usd([12755, 30586, 6383127, 633936],[1,1.1,0.8,0.01])
grand_total_usd(sub_totals_in_usd)

5159240.5600000005

# Los Map, Reduce y Filter de Python


* `map` map(function, iterables)
* `reduce` from functools import reduce(funtion, iterable, initial_value)
* `filter` filter(function, iterable)

¡No devuelven listas! ¿Por qué?


------------------

# Analisis de Datos con mapReduce

Vamos a crear un histograma de la frecuencia de palabras en un extracto de Alicia en el Pais de las Maravillas. Queremso saber con qué frecuencia aparece cada palabra.

Es un caso típico del uso de mapReduce.

## Plan

1. Tokenizar el texto
1. Limpiar de símbolos raros
1. Eliminar [stopwords](https://www.kaggle.com/datasets/rowhitswami/stopwords)
1. Normalizar la representación
1. Calcular el histograma
1. Normalizar el historgrama
1. Imprimirlo

In [ ]:
text = """
“I’m sure those are not the right words,” said poor Alice, and her eyes filled with tears again as she went on, “I must be Mabel after all, and I shall have to go and live in that poky little house, and have next to no toys to play with, and oh! ever so many lessons to learn! No, I’ve made up my mind about it; if I’m Mabel, I’ll stay down here! It’ll be no use their putting their heads down and saying ‘Come up again, dear!’ I shall only look up and say ‘Who am I then? Tell me that first, and then, if I like being that person, I’ll come up: if not, I’ll stay down here till I’m somebody else’—but, oh dear!” cried Alice, with a sudden burst of tears, “I do wish they would put their heads down! I am so very tired of being all alone here!”

As she said this she looked down at her hands, and was surprised to see that she had put on one of the Rabbit’s little white kid gloves while she was talking. “How can I have done that?” she thought. “I must be growing small again.” She got up and went to the table to measure herself by it, and found that, as nearly as she could guess, she was now about two feet high, and was going on shrinking rapidly: she soon found out that the cause of this was the fan she was holding, and she dropped it hastily, just in time to avoid shrinking away altogether.

“That was a narrow escape!” said Alice, a good deal frightened at the sudden change, but very glad to find herself still in existence; “and now for the garden!” and she ran with all speed back to the little door: but, alas! the little door was shut again, and the little golden key was lying on the glass table as before, “and things are worse than ever,” thought the poor child, “for I never was so small as this before, never! And I declare it’s too bad, that it is!”

As she said these words her foot slipped, and in another moment, splash! she was up to her chin in salt water. Her first idea was that she had somehow fallen into the sea, “and in that case I can go back by railway,” she said to herself. (Alice had been to the seaside once in her life, and had come to the general conclusion, that wherever you go to on the English coast you find a number of bathing machines in the sea, some children digging in the sand with wooden spades, then a row of lodging houses, and behind them a railway station.) However, she soon made out that she was in the pool of tears which she had wept when she was nine feet high.

“I wish I hadn’t cried so much!” said Alice, as she swam about, trying to find her way out. “I shall be punished for it now, I suppose, by being drowned in my own tears! That will be a queer thing, to be sure! However, everything is queer to-day.”

Just then she heard something splashing about in the pool a little way off, and she swam nearer to make out what it was: at first she thought it must be a walrus or hippopotamus, but then she remembered how small she was now, and she soon made out that it was only a mouse that had slipped in like herself.

“Would it be of any use, now,” thought Alice, “to speak to this mouse? Everything is so out-of-the-way down here, that I should think very likely it can talk: at any rate, there’s no harm in trying.” So she began: “O Mouse, do you know the way out of this pool? I am very tired of swimming about here, O Mouse!” (Alice thought this must be the right way of speaking to a mouse: she had never done such a thing before, but she remembered having seen in her brother’s Latin Grammar, “A mouse—of a mouse—to a mouse—a mouse—O mouse!”) The Mouse looked at her rather inquisitively, and seemed to her to wink with one of its little eyes, but it said nothing.

“Perhaps it doesn’t understand English,” thought Alice; “I daresay it’s a French mouse, come over with William the Conqueror.” (For, with all her knowledge of history, Alice had no very clear notion how long ago anything had happened.) So she began again: “Où est ma chatte?” which was the first sentence in her French lesson-book. The Mouse gave a sudden leap out of the water, and seemed to quiver all over with fright. “Oh, I beg your pardon!” cried Alice hastily, afraid that she had hurt the poor animal’s feelings. “I quite forgot you didn’t like cats.”

“Not like cats!” cried the Mouse, in a shrill, passionate voice. “Would you like cats if you were me?”

“Well, perhaps not,” said Alice in a soothing tone: “don’t be angry about it. And yet I wish I could show you our cat Dinah: I think you’d take a fancy to cats if you could only see her. She is such a dear quiet thing,” Alice went on, half to herself, as she swam lazily about in the pool, “and she sits purring so nicely by the fire, licking her paws and washing her face—and she is such a nice soft thing to nurse—and she’s such a capital one for catching mice—oh, I beg your pardon!” cried Alice again, for this time the Mouse was bristling all over, and she felt certain it must be really offended. “We won’t talk about her any more if you’d rather not.”

“We indeed!” cried the Mouse, who was trembling down to the end of his tail. “As if I would talk on such a subject! Our family always hated cats: nasty, low, vulgar things! Don’t let me hear the name again!”

“I won’t indeed!” said Alice, in a great hurry to change the subject of conversation. “Are you—are you fond—of—of dogs?” The Mouse did not answer, so Alice went on eagerly: “There is such a nice little dog near our house I should like to show you! A little bright-eyed terrier, you know, with oh, such long curly brown hair! And it’ll fetch things when you throw them, and it’ll sit up and beg for its dinner, and all sorts of things—I can’t remember half of them—and it belongs to a farmer, you know, and he says it’s so useful, it’s worth a hundred pounds! He says it kills all the rats and—oh dear!” cried Alice in a sorrowful tone, “I’m afraid I’ve offended it again!” For the Mouse was swimming away from her as hard as it could go, and making quite a commotion in the pool as it went.

So she called softly after it, “Mouse dear! Do come back again, and we won’t talk about cats or dogs either, if you don’t like them!” When the Mouse heard this, it turned round and swam slowly back to her: its face was quite pale (with passion, Alice thought), and it said in a low trembling voice, “Let us get to the shore, and then I’ll tell you my history, and you’ll understand why it is I hate cats and dogs.”

It was high time to go, for the pool was getting quite crowded with the birds and animals that had fallen into it: there were a Duck and a Dodo, a Lory and an Eaglet, and several other curious creatures. Alice led the way, and the whole party swam to the shore.

"""

# Implementación

1. A mano
1. Con herramientas específicas y frameworks de data science. [Pandas](https://pandas.pydata.org/)
1. Con [Apache Spark](https://spark.apache.org/)